In [1]:
import os
import urllib
import numpy as np
from os import listdir
from os.path import isfile, join
from scipy import misc
import random

import tensorflow as tf
from tensorflow.python.ops.constant_op import constant
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np

In [2]:
import random as rand

def extract_images_labels(folder_name):
    images = [f for f in listdir(folder_name) if isfile(join(folder_name, f)) and f.endswith('png')]
    rand.shuffle(images)
    lbs = np.zeros((len(images), 10))
    counter = 0
    for image in images:
        vals = np.zeros(10)
        vals[int(image[0])] = 1
        lbs[counter,:] = vals
        counter += 1
    return images, lbs;

In [3]:
folder_name = "/home/agah/TF/CSE253Project/spoken_numbers"

In [4]:
img_paths, img_labels = extract_images_labels(folder_name);

assert len(img_paths) == len(img_labels)
index = random.randint(0,len(img_paths)-1)
print index
print img_paths[index]
print img_labels[index]

total_number_of_images = len(img_paths);
print "Total # of images", total_number_of_images

3797
2_Samantha_120.wav.png
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
Total # of images 4684


In [5]:
class DataSet(object):
    def __init__(self, images, labels):
        assert len(images) == len(labels), (
          "images.shape: %s labels.shape: %s" % (len(images), len(labels)))
        self._num_examples = len(images)

        self._images = np.asarray(images)
        self._labels = np.asarray(labels)
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images
    
    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, fake_data=False):
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._images[start:end], self._labels[start:end]

In [6]:
tr_size = int(0.95 * total_number_of_images)
te_size = total_number_of_images - tr_size

In [7]:
def read_data_sets(train_dir):
    class DataSets(object):
        pass
    
    data_sets = DataSets()
    images, labels = extract_images_labels(train_dir)

    train_images = images[:tr_size]
    train_labels = labels[:tr_size]
    
    test_images = images[tr_size:]
    test_labels = labels[tr_size:]

    print len(train_images), len(train_labels)
    print len(test_images), len(test_labels)

    data_sets.train = DataSet(train_images, train_labels)
    data_sets.test = DataSet(test_images, test_labels)
    
    return data_sets

In [8]:
data = read_data_sets(folder_name)

print("data returned")

4449 4449
235 235
data returned


In [9]:
'''
Every image row is a sequence of pixels, so 512 sequences of 512 steps for every input
'''

learning_rate = 0.001
training_iters = 100 * 1280
batch_size = 128
display_step = 10

In [10]:
# Network Parameters
n_input = 512 # 512*512 images
n_steps = 512 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # Total classes (0-9 digits)

In [11]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])

# Tensorflow LSTM cell requires 2x n_hidden length (state & cell)
istate_fw = tf.placeholder("float", [None, 2*n_hidden])
istate_bw = tf.placeholder("float", [None, 2*n_hidden])
y = tf.placeholder("float", [None, n_classes])

In [12]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'hidden': tf.Variable(tf.random_normal([n_input, 2*n_hidden])),
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([2*n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [13]:
def BiRNN(_X, _istate_fw, _istate_bw, _weights, _biases, _batch_size, _seq_len):

    # BiRNN requires to supply sequence_length as [batch_size, int64]
    # Note: Tensorflow 0.6.0 requires BiRNN sequence_length parameter to be set
    # For a better implementation with latest version of tensorflow, check below
    _seq_len = tf.fill([_batch_size], constant(_seq_len, dtype=tf.int64))

    print("before transpose")
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) # (n_steps*batch_size, n_input)
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']

    print("Done matmul")
    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    print("before outputs")

    # Get lstm cell output
    outputs = rnn.bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, _X,
                                            initial_state_fw=_istate_fw,
                                            initial_state_bw=_istate_bw,
                                            sequence_length=_seq_len)
    print("before return");
    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [14]:
print("Calling BiRNN")
pred = BiRNN(x, istate_fw, istate_bw, weights, biases, batch_size, n_steps)
print("Calling BiRNN Done")

Calling BiRNN
before transpose
Done matmul
before outputs
before return
Calling BiRNN Done


In [15]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

In [16]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [17]:
# Initializing the variables
init = tf.initialize_all_variables()

In [18]:
def get_images(image_paths):
    rets = np.zeros((len(image_paths), 512, 512))
    counter = 0
    for image in image_paths:
        img_data = misc.imread(join(folder_name, image))
        rets[counter][:] = img_data
        counter += 1

    images = rets.reshape(rets.shape[0], rets.shape[1] * rets.shape[2])
    
    # Convert from [0, 255] -> [0.0, 1.0].
    images = images.astype(np.float32)
    images = np.multiply(images, 1.0 / 255.0)
    
    return images

In [19]:
print("Launching the graph")
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs_paths, batch_ys = data.train.next_batch(batch_size)
        batch_xs = get_images(batch_xs_paths)
        
        # Reshape data to get 28 seq of 28 elements
        batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))
        
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                       istate_fw: np.zeros((batch_size, 2*n_hidden)),
                                       istate_bw: np.zeros((batch_size, 2*n_hidden))})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys,
                                                istate_fw: np.zeros((batch_size, 2*n_hidden)),
                                                istate_bw: np.zeros((batch_size, 2*n_hidden))})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,
                                             istate_fw: np.zeros((batch_size, 2*n_hidden)),
                                             istate_bw: np.zeros((batch_size, 2*n_hidden))})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
        
    print "Optimization Finished!"
    
#     Calculate test accuracy
    test_len = 128
    test_images = get_images(data.test.images[:test_len])
    test_data = test_images.reshape((-1, n_steps, n_input))
    test_label = data.test.labels[:test_len]
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                                             istate_fw: np.zeros((test_len, 2*n_hidden)),
                                                             istate_bw: np.zeros((test_len, 2*n_hidden))})

Launching the graph
Iter 1280, Minibatch Loss= 3.299364, Training Accuracy= 0.23438
Iter 2560, Minibatch Loss= 2.495505, Training Accuracy= 0.32812
Iter 3840, Minibatch Loss= 2.310158, Training Accuracy= 0.35156
Iter 5120, Minibatch Loss= 1.853615, Training Accuracy= 0.43750
Iter 6400, Minibatch Loss= 1.596397, Training Accuracy= 0.48438
Iter 7680, Minibatch Loss= 1.722748, Training Accuracy= 0.47656
Iter 8960, Minibatch Loss= 1.579008, Training Accuracy= 0.45312
Iter 10240, Minibatch Loss= 1.313428, Training Accuracy= 0.56250
Iter 11520, Minibatch Loss= 1.441493, Training Accuracy= 0.48438
Iter 12800, Minibatch Loss= 1.558009, Training Accuracy= 0.48438
Iter 14080, Minibatch Loss= 1.495282, Training Accuracy= 0.50000
Iter 15360, Minibatch Loss= 1.310763, Training Accuracy= 0.55469
Iter 16640, Minibatch Loss= 1.475117, Training Accuracy= 0.47656
Iter 17920, Minibatch Loss= 0.902650, Training Accuracy= 0.68750
Iter 19200, Minibatch Loss= 1.008525, Training Accuracy= 0.65625
Iter 20480, 